In [14]:


import numpy as np

def midi2nmat(filename):
    # % USE KEN SCHUTTE'S FUNCTIONS
    m = readmidi(filename)
    n = midiInfo(m, 0)
    [tempos, tempos_time] = getTempoChanges(m)
    # % CONVERT OUTPUT INTO MIDI TOOLBOX NMAT FORMAT
    nmat[:,6].lvalue = n[:,5]
    nmat[:,7].lvalue = n[:,6] - n[:,5]  # duration
    nmat[:,3].lvalue = n[:,2]
    nmat[:,4].lvalue = n[:,3]
    nmat[:,5].lvalue = n[:,4]
    # % CHECK IF MULTIPLE TEMPI
    if not tempos:
        tempos = 500000

    # some files may not contain the tempo, use default in that case
    if len(tempos) > 1:
        # here for diff tempi....
        print('WARNING: Multiple tempi detected, simple recoding of timing (seconds, not beats!)')
        print(str(60/(tempos/1000000)))
        tc_ind = 0
        starttime = 0

        for i in range(1,len(tempos_time) - 1):
            tempo_begin = (tempos_time(i + 1) - tempos_time(i)) / m.ticks_per_quarter_note * tempos(i) / 1000000
            tempo_begin = tempo_begin + starttime
            starttime = tempo_begin
            tc_ind = [tc_ind, tempo_begin]

        nmat2 = nmat


        for i in range(1,len(tc_ind)-1):
            if i == len(tc_ind):
                time_index = nmat[:, 6] >= tc_ind(i)
            else:
                time_index = nmat[:, 6] >= tc_ind(i)  # & nmat(:,6)<=tc_ind(i+1);
            timeratio = tempos(i) / tempos(1)
            # realign timing after tempo changes
            tmp1 = nmat2(time_index)
            tmp2 = n(time_index, 5) * timeratio
            tmp2 = tmp2(1)
            realign = tmp1(1, 6) - tmp2  # previous onset time
            nmat2(time_index, 6).lvalue = (n(time_index, 5) * timeratio) + realign
            nmat2(time_index, 7).lvalue = nmat2(time_index, 7) * timeratio

            nmat[:, 1].lvalue = n[:, 5] / (tempos(1) / 1000000)  # beats are not modified according to tempi
            nmat[:, 2].lvalue = n[:, 6] - n[:, 5]) / (tempos(1) / 1000000
    else:
        nmat[:, 1].lvalue = n[:, 5] / (tempos(1) / 1000000)
        nmat[:, 2].lvalue = n[:, 6] - n[:, 5]) / (tempos(1) / 1000000

    [nmat, n] = midi2mat('/Users/EJ/dev/vmo/bin/files/midi/parker_omnibook/blues_for_alice.mid')

    # postprocessing...
    # IF CHANNEL IS EMPTY
    if nmat[:, 3] == 0:
        nmat[:, 3].lvalue = 1
    # FUNCTIONS
    def midiInfo(midi, outputFormat, tracklist):

        if nargin < 3:
            tracklist = []
            if nargin < 2:
                outputFormat = 1
        if not tracklist:
            tracklist = range(1,len(midi.track))


    [tempos, tempos_time] = getTempoChanges(midi)
    current_tempo = 500000  # default tempo
    fid = -1
    if str.isalpha(outputFormat):
        fid = open(outputFormat, 'w')

    endtime = -1
    # each row:
    #  1     2    3  4   5  6  7       8
    # [track chan nn vel t1 t2 msgNum1 msgNum2]
    Notes = np.zeros(0, 8)

    for i in range(1,len(tracklist)):
        tracknum = tracklist(i)
        cumtime = 0
        seconds = 0
        Msg = cell(0)
        Msg(1, 1).lvalue = 'chan'
        Msg(1, 2).lvalue = 'deltatime'
        Msg(1, 3).lvalue = 'time'
        Msg(1, 4).lvalue = 'name'
        Msg(1, 5).lvalue = 'data'
        runnro = 1
        for msgNum in range(1,len(midi.track(tracknum).messages)):
            currMsg = midi.track(tracknum).messages(msgNum)
            midimeta = currMsg.midimeta
            deltatime = currMsg.deltatime
            data = currMsg.data
            type = currMsg.type
            chan = currMsg.chan
            cumtime = cumtime + deltatime
            # current_tempo/midi.ticks_per_quarter_note;
            # debug = deltatime*1e-6*current_tempo/midi.ticks_per_quarter_note;
            seconds = seconds + deltatime * 1e-6 * current_tempo / midi.ticks_per_quarter_note
            [mx, ind] = max(find(cumtime >= tempos_time))
            # % % ADDED BY TE 1.1.2008
            if not ind:

            else:
                current_tempo = tempos(ind)
            # find start/stop of notes:
            #    if (strcmp(name,'Note on') && (data(2)>0))
            # note on with vel>0:
            if (midimeta == 1 and type == 144 and data(2) > 0):
                # note on:
                if not Notes:
                    Notes[1,:].lvalue = [tracknum, chan, data(1), data(2), seconds, 0, runnro - 1]
                    runnro = runnro + 1
                else:
                    #       Notes(end+1,:) = [tracknum chan data(1) data(2) seconds 0 msgNum -1]
                    Notes[end+1,:].lvalue = [tracknum, chan, data(1), data(2), seconds, 0, runnro - 1]
                    runnro = runnro + 1
            #    elseif ((strcmp(name,'Note on') && (data(2)==0)) || strcmp(name,'Note off'))
            # note on with vel==0 or note off:
            elif (midimeta == 1 and ((type == 144 and data(2) == 0) or type == 128)):
                # note off:
                #      % find index, wther tr,chan,and nn match, and not complete
                ind = find(((Notes[:,1] == tracknum) + (Notes[:,2] == chan) + (Notes[:,3] == data(1)) + (Notes[:,8] == -1)) == 4)

                if (len(ind) == 0):
                    print 'ending non-open note?'
                elif (len(ind) > 1):
                    print 'warning: found multiple matches in endNote, taking first...'
                    ind = ind(1)
                # set info on ending:
                Notes(ind, 6).lvalue = seconds
                Notes(ind, 8).lvalue = msgNum
            # end of track:
            elif (midimeta == 0 and type == 47):
                endtime = seconds
            else:
                print 'two "end of track" messages?'
                endtime(end + 1).lvalue = seconds
            # we could check to make sure it ends with
            #  'end of track'
            if (outputFormat != 0):
                # get some specific descriptions:
                name = str(type)
                dataStr = str(data)
                if not chan:
                    Msg(msgNum, 1).lvalue = '-'
                else:
                    Msg(msgNum, 1).lvalue = str(chan)

                Msg(msgNum, 2).lvalue = str(deltatime)
                Msg(msgNum, 3).lvalue = formatTime(seconds)
                if (midimeta == 0):
                    Msg(msgNum, 4).lvalue = 'meta'
                else:
                    Msg(msgNum, 4).lvalue = ''
                [name, dataStr] = getMsgInfo(midimeta, type, data)
                Msg(msgNum, 5).lvalue = name
                Msg(msgNum, 6).lvalue = dataStr
        if (outputFormat != 0):
            print(Msg, tracknum, fid)
    # make this an option!!!
    # - I'm not sure why it's needed...
    # remove start silence:
    # _ removed by TE
    # _first_t = min(Notes(:,5));
    # _Notes(:,5) = Notes(:,5) - first_t;
    # _Notes(:,6) = Notes(:,6) - first_t;
    # sort Notes by start time:
    [junk, ord] = sort(Notes[:,5])
    Notes = Notes[ord,:]
    if (fid != -1):
        close(fid)



def printTrackInfo(Msg=None, tracknum=None, fid=None):


    # make cols same length instead of just using \t
    for i in range(1,size(Msg, 2):
        maxLen(i).lvalue = 0
    for j in range(1,size(Msg, 1)):
        if (len(Msg(j, i)) > maxLen(i)):
    maxLen(i).lvalue = len(Msg(j, i))


    s = ''
    s = [s, sprintf('--------------------------------------------------\\n')]
    s = [s, sprintf('Track %d\\n'), tracknum)]
    s = [s, sprintf('--------------------------------------------------\\n')]

    if (fid == -1):
        disp(s)
    else:
        fprintf(fid,'%s', s)


    for i in range(1,size(Msg, 1)):
        line = ''
        for j in range(1,size(Msg, 2)):
            sp = repmat(' ', 1, 5 + maxLen(j) - len(Msg(i, j)))
            m = Msg(i, j)
            m = m[:].cT        # ensure column vector
            #    line = [line Msg{i,j} sp];
            line = [line, m, sp]

        if (fid == -1):
            disp(line)
        else:
            fprintf(fid, '%s\\n', line)




def formatTime(seconds=None):

    minutes = floor(seconds / 60)
    secs = seconds - 60 * minutes

    s = sprintf('%d:%2.3f'), minutes, secs)



    def getMsgInfo(midimeta=None, type=None, data=None):

        # meta events:
        if (midimeta == 0):
            if (type == 0):
                name = 'Sequence Number'
                len = 2
                dataStr = str(data)

            elif (type == 1):
                name = 'Text Events'
                len = -1
                dataStr = chr(data)

            elif (type == 2):
                name = 'Copyright Notice'
                len = -1
                dataStr = chr(data)

            elif (type == 3):
                name = 'Sequence/Track Name'
                len = -1
                dataStr = chr(data)

            elif (type == 4):
                name = 'Instrument Name'
                len = -1
                dataStr = chr(data)

            elif (type == 5):
                name = 'Lyric'
                len = -1
                dataStr = chr(data)

            elif (type == 6):
                name = 'Marker'
                len = -1
                dataStr = chr(data)

            elif (type == 7):
                name = 'Cue Point'
                len = -1
                dataStr = chr(data)

            elif (type == 32):
                name = 'MIDI Channel Prefix'
                len = 1
                dataStr = str(data)

            elif (type == 47):
                name = 'End of Track'
                len = 0
                dataStr = ''

            elif (type == 81):
                name = 'Set Tempo'
                len = 3

                val = data(1) * 16 ** 4 + data(2) * 16 ** 2 + data(3)
                dataStr = ['microsec per quarter note: ', str(val)]

            elif (type == 84):
                name = 'SMPTE Offset'
                len = 5

                if size(data) == 5:
                    dataStr = ['[hh mm ss fr ff]=', str(data)]
                else:
                    dataStr = ['[hh mm ss fr ff]=', str(data.cT)]

            elif (type == 88):
                name = 'Time Signature'
                len = 4

                dataStr = [data(1), '/', data(2), ', clock ticks and notated 32nd notes=', data(3), '/', str(data(4))]
            elif (type == 89):
                name = 'Key Signature'
                len = 2

                # num sharps/flats (flats negative)
                if (data(1) >= 0):
                    #       1   2    3    4   5     6    7
                    ss = ['C', 'G', 'D', 'A', 'E', 'B', 'F#', 'C#']
                    if data(1) > 7:
                        dataStr = 'C'               # ADDED BY TE 1.1.2008
                    else:
                        dataStr = ss(data(1) + 1)


                else:
                    #    1   2    3    4   5     6    7
                    ss = ['F', 'Bb', 'Eb', 'Ab', 'Db', 'Gb', 'Cb']
                    dataStr = ss(abs(data(1)))

                if (data(2) == 0):
                    dataStr = [dataStr, ' Major']
                else:
                    dataStr = [dataStr, ' Minor']


            elif (type == 89):
                name = 'Sequencer-Specific Meta-event'
                len = -1

                dataStr = chr(data)
                # !! last two conflict...

            else:
                name = ['UNKNOWN META EVENT: ', str(type)]
                dataStr = str(data)


                # meta 0x21 = MIDI port number, length 1 (? perhaps)
        else:

            # channel voice messages:
            #  (from event byte with chan removed, eg 0x8n -> 0x80 = 128 for
            #  note off)
            if (type == 128):
                name = 'Note off'
                len = 2
                dataStr = ['nn=', str(data(1)), '  vel=', str(data(2))]

            elif (type == 144):
                name = 'Note on'
                len = 2
                dataStr = ['nn=', str(data(1)), '  vel=', str(data(2))]

            elif (type == 160):
                name = 'Polyphonic Key Pressure'
                len = 2
                dataStr = ['nn=', str(data(1)), '  vel=', str(data(2))]

            elif (type == 176):
                name = 'Controller Change'
                len = 2
                dataStr = ['ctrl=', controllers(data(1)), '  value=', str(data(2))]

            elif (type == 192):
                name = 'Program Change'
                len = 1
                dataStr = ['instr=', str(data)]

            elif (type == 208):
                name = 'Channel Key Pressure'
                len = 1
                dataStr = ['vel=', str(data)]

            elif (type == 224):
                name = 'Pitch Bend'
                len = 2

                val = data(1) + data(2) * 256
                val = base2dec('2000', 16) - val
                dataStr = ['change=', str(val), '?']

                # channel mode messages:
                #  ... unsure about data for these... (do some have a data byte and
                #  others not?)
                #
                # 0xC1 .. 0xC8
            elif (type == 193):
                name = 'All Sounds Off'
                dataStr = str(data)

            elif (type == 194):
                name = 'Reset All Controllers'
                dataStr = str(data)

            elif (type == 195):
                name = 'Local Control'
                dataStr = str(data)

            elif (type == 196):
                name = 'All Notes Off'
                dataStr = str(data)

            elif (type == 197):
                name = 'Omni Mode Off'
                dataStr = str(data)

            elif (type == 198):
                name = 'Omni Mode On'
                dataStr = str(data)

            elif (type == 199):
                name = 'Mono Mode On'
                dataStr = str(data)

            elif (type == 200):
                name = 'Poly Mode On'
                dataStr = str(data)


                # sysex, F0->F7
            elif (type == 240):
                name = 'Sysex 0xF0'
                dataStr = str(data)

            elif (type == 241):
                name = 'Sysex 0xF1'
                dataStr = str(data)

            elif (type == 242):
                name = 'Sysex 0xF2'
                dataStr = str(data)

            elif (type == 243):
                name = 'Sysex 0xF3'
                dataStr = str(data)

            elif (type == 244):
                name = 'Sysex 0xF4'
                dataStr = str(data)

            elif (type == 245):
                name = 'Sysex 0xF5'
                dataStr = str(data)

            elif (type == 246):
                name = 'Sysex 0xF6'
                dataStr = str(data)

            elif (type == 247):
                name = 'Sysex 0xF7'
                dataStr = str(data)


                # realtime
                # (i think have no data..?)
            elif (type == 248):
                name = 'Real-time 0xF8 - Timing clock'
                dataStr = str(data)

            elif (type == 249):
                name = 'Real-time 0xF9'
                dataStr = str(data)

            elif (type == 250):
                name = 'Real-time 0xFA - Start a sequence'
                dataStr = str(data)

            elif (type == 251):
                name = 'Real-time 0xFB - Continue a sequence'
                dataStr = str(data)

            elif (type == 252):
                name = 'Real-time 0xFC - Stop a sequence'
                dataStr = str(data)

            elif (type == 253):
                name = 'Real-time 0xFD'
                dataStr = str(data)

            elif (type == 254):
                name = 'Real-time 0xFE'
                dataStr = str(data)

            elif (type == 255):
                name = 'Real-time 0xFF'
                dataStr = str(data)



            else:
                name = ['UNKNOWN MIDI EVENT: ', str(type)]
                dataStr = str(data)



    def controllers(n=None):
        if (n == 1):
            s = 'Mod Wheel'

        elif (n == 2):
            s = 'Breath Controllery'

        elif (n == 4):
            s = 'Foot Controller'

        elif (n == 5):
            s = 'Portamento Time'

        elif (n == 6):
            s = 'Data Entry MSB'

        elif (n == 7):
            s = 'Volume'

        elif (n == 8):
            s = 'Balance'

        elif (n == 10):
            s = 'Pan'

        elif (n == 11):
            s = 'Expression Controller'

        elif (n == 16):
            s = 'General Purpose 1'

        elif (n == 17):
            s = 'General Purpose 2'

        elif (n == 18):
            s = 'General Purpose 3'

        elif (n == 19):
            s = 'General Purpose 4'

        elif (n == 64):
            s = 'Sustain'

        elif (n == 65):
            s = 'Portamento'

        elif (n == 66):
            s = 'Sustenuto'

        elif (n == 67):
            s = 'Soft Pedal'

        elif (n == 69):
            s = 'Hold 2'

        elif (n == 80):
            s = 'General Purpose 5'

        elif (n == 81):
            s = 'Temp Change (General Purpose 6)'

        elif (n == 82):
            s = 'General Purpose 7'

        elif (n == 83):
            s = 'General Purpose 8'

        elif (n == 91):
            s = 'Ext Effects Depth'

        elif (n == 92):
            s = 'Tremelo Depthy'

        elif (n == 93):
            s = 'Chorus Depth'

        elif (n == 94):
            s = 'Detune Depth (Celeste Depth)'

        elif (n == 95):
            s = 'Phaser Depth'

        elif (n == 96):
            s = 'Data Increment (Data Entry +1)'

        elif (n == 97):
            s = 'Data Decrement (Data Entry -1)'

        elif (n == 98):
            s = 'Non-Registered Param LSB'

        elif (n == 99):
            s = 'Non-Registered Param MSB'

        elif (n == 100):
            s = 'Registered Param LSB'

        elif (n == 101):
            s = 'Registered Param MSB'

        else:
            s = 'UNKNOWN CONTROLLER'

            #Channel mode message values
            #Reset All Controllers      79      121     Val ??
            #Local Control      7A      122     Val 0 = off, 7F (127) = on
            #All Notes Off      7B      123     Val must be 0
            #Omni Mode Off      7C      124     Val must be 0
            #Omni Mode On       7D      125     Val must be 0
            #Mono Mode On       7E      126     Val = # of channels, or 0 if # channels equals # voices in receiver
            #Poly Mode On       7F      127     Val must be 0


    def getTempoChanges(midi=None):
        # [tempos,tempos_time]=getTempoChanges(midi)
        #
        # input: a midi struct from readmidi.m
        # output:
        #  tempos = tempo values indexed by tempos_time
        #    tempos_time is in units of ticks
        #
        # should tempo changes effect across tracks? across channels?
        #
        #---------------------------------------------------------------
        # Subversion Revision: 14 (2006-01-24)
        #
        # This software can be used freely for non-commerical use.
        # Visit http://www.kenschutte.com/software for more
        #   documentation, copyright info, and updates.
        #---------------------------------------------------------------


        tempos = []
        tempos_time = []
        #tempos_index = [];
        for i in range(1,len(midi.track)):
            cumtime = 0
            for j in range(1,len(midi.track(i).messages)):
                cumtime = cumtime + midi.track(i).messages(j).deltatime
                #    if (strcmp(midi.track(i).messages(j).name,'Set Tempo'))
                if (midi.track(i).messages(j).midimeta == 0 and midi.track(i).messages(j).type == 81):

                    tempos_time(end + 1).lvalue = cumtime
                    d = midi.track(i).messages(j).data
                    tempos(end + 1).lvalue = d(1) * 16 ** 4 + d(2) * 16 ** 2 + d(3)





    def readmidi(filename=None, rawbytes=None):
        # midi = readmidi(filename, rawbytes)
        # midi = readmidi(filename)
        #
        # Read MIDI file and store in a Matlab structure
        # (use midiInfo.m to see structure detail)
        #
        # Inputs:
        #  filename - input MIDI file
        #  rawbytes - 0 or 1: Include raw bytes in structure
        #             This info is redundant, but can be
        #             useful for debugging. default=0
        #
        #---------------------------------------------------------------
        # Subversion Revision: 14 (2006-12-03)
        #
        # This software can be used freely for non-commerical use.
        # Visit http://www.kenschutte.com/software for more
        #   documentation, copyright info, and updates.
        #---------------------------------------------------------------



        if (nargin < 2):
            rawbytes = 0


        fid = open(filename)
        #[A count] = fread(fid,'char');
        [A, count] = np.fromfile(fid, str('uint8'))
        file.close(fid)

        midi.filename = filename
        if (rawbytes):
            midi.rawbytes_all = A



            # realtime events: status: [F8, FF].  no data bytes
            #clock, undefined, start, continue, stop, undefined, active
            #sensing, systerm reset

            # file consists of "header chunk" and "track chunks"
            #   4B  'MThd' (header) or 'MTrk' (track)
            #   4B  32-bit unsigned int = number of bytes in chunk, not
            #       counting these first 8


            # HEADER CHUNK --------------------------------------------------------
            # 4B 'Mthd'
            # 4B length
            # 2B file format
            #    0=single track, 1=multitrack synchronous, 2=multitrack asynchronous
            #    Synchronous formats start all tracks at the same time, while asynchronous formats can start and end any track at any time during the score.
            # 2B track cout (must be 1 for format 0)
            # 2B num delta-time ticks per quarter note
            #


            #error(mstring('File does not begin with header ID (MThd)'))


    header_len = decode_int(A[5:8])
    if (header_len == 6):
        print 'okay'
    else:
        print 'Header length != 6 bytes.'


    format = decode_int(A[9:10])
    if (format == 0 or format == 1 or format == 2):
        midi.format = format
    else:
        print 'Format does not equal 0,1,or 2'


    num_tracks = decode_int(A[11:12])
    if (format == 0 and num_tracks != 1):
        print 'File is format 0, but num_tracks != 1'


    time_unit = decode_int(A[13:14])
    if (bitand(time_unit, 2 ** 15) == 0):
        midi.ticks_per_quarter_note = time_unit
    else:
        print 'Header: SMPTE time format found - not currently supported'


    if (rawbytes):
        midi.rawbytes_header = A[1:14]


    # end header parse ----------------------------------------------------






    # BREAK INTO SEPARATE TRACKS ------------------------------------------
    # midi.track(1).data = [byte byte byte ...];
    # midi.track(2).date = ...
    # ...
    #
    # Track Chunks---------
    # 4B 'MTrk'
    # 4B length (after first 8B)
    #
    ctr = 15
    i = 0

    while i < num_tracks:
        print ['Track ', str(i), ' does not begin with track ID=MTrk']

    ctr = ctr + 4

    track_len = decode_int(A[ctr:ctr + 3])
    ctr = ctr + 4

    # have track.rawbytes hold initial 8B also...
    track_rawbytes(i).lvalue = A[(ctr - 8):(ctr + track_len - 1)]

    if (rawbytes):
        midi.track(i).rawbytes_header.lvalue = A[ctr - 8:ctr - 1]


    ctr = ctr + track_len

    # ----------------------------------------------------------------------






    # Events:
    #  - meta events: start with 'FF'
    #  - MIDI events: all others

    # MIDI events:
    #  optional command byte + 0,1,or 2 bytes of parameters
    #  "running mode": command byte omitted.
    #
    # all midi command bytes have MSB=1
    # all data for inside midi command have value <= 127 (ie MSB=0)
    # -> so can determine running mode
    #
    # meta events' data may have any values (meta events have to set
    # len)
    #



    # 'Fn' MIDI commands:
    #  no chan. control the entire system
    #F8 Timing Clock
    #FA start a sequence
    #FB continue a sequence
    #FC stop a sequence

    # Meta events:
    #  1B 0xFF
    #  1B event type
    #  1B length of additional data
    #  ?? additional data
    #


    # "channel mode messages"
    # have same code as "control change": 0xBn
    #  but uses reserved controller numbers 120-127
    #


    #Midi events consist of an optional command byte
    # followed by zero, one or two bytes of parameters.
    # In running mode, the command can be omitted, in
    # which case the last MIDI command specified is
    # assumed.  The first bit of a command byte is 1,
    # while the first bit of a parameter is always 0.
    #   In addition, the last 4 bits of a command
    #   indicate the channel to which the event should
    #   be sent; therefore, there are 6 possible
    #   commands (really 7, but we will discuss the x'Fn'
    #   commands later) that can be specified.  They are:


    # parse tracks -----------------------------------------
    for i in range(1,num_tracks):

        track = track_rawbytes(i)

        if (rawbytes):
            midi.track(i).rawbytes.lvalue = track

        msgCtr = 1
        ctr = 9# first 8B were MTrk and length
        while (ctr < len(track_rawbytes(i))):

            clear("currMsg")
            currMsg.used_running_mode = 0
            # note:
            #  .used_running_mode is necessary only to
            #  be able to reconstruct a file _exactly_ from
            #  the 'midi' structure.  this is helpful for
            #  debugging since write(read(filename)) can be
            #  tested for exact replication...
            #

            ctr_start_msg = ctr

            [deltatime, ctr] = decode_var_length(track, ctr)

            # ?
            #if (rawbytes)
            #  currMsg.rawbytes_deltatime = track(ctr_start_msg:ctr-1);
            #end

            # deltaime must be 1-4 bytes long.
            # could check here...


            # CHECK FOR META EVENTS ------------------------
            # 'FF'
            if track(ctr) == 255:

                type = track(ctr + 1)

                ctr = ctr + 2

                # get variable length 'length' field
                [len, ctr] = decode_var_length(track, ctr)

                # note: some meta events have pre-determined lengths...
                #  we could try verifiying they are correct here.

                thedata = track[ctr:ctr + len - 1]
                chan = []

                ctr = ctr + len

                midimeta = 0

            else:
                midimeta = 1
                # MIDI EVENT ---------------------------




                # check for running mode:
                if (track(ctr) < 128):

                    # make it re-do last command:
                    #ctr = ctr - 1;
                    #track(ctr) = last_byte;
                    currMsg.used_running_mode = 1

                    B = last_byte
                    nB = track(ctr)            # ?

                else:

                    B = track(ctr)
                    nB = track(ctr + 1)

                    ctr = ctr + 1

                # nibbles:
                #B  = track(ctr);
                #nB = track(ctr+1);


                Hn = bitshift(B, -4)
                Ln = bitand(B, 15)

                chan = []

                msg_type = midi_msg_type(B, nB)

                # DEBUG:
                if (i == 2):
                    if (msgCtr == 1):
                        disp(msg_type)




                __switch_0__ = msg_type
                if 0:
                    pass

                elif __switch_0__ == 'channel_mode':

                    # UNSURE: if all channel mode messages have 2 data byes (?)
                    type = bitshift(Hn, 4) + (nB - 120 + 1)
                    thedata = track[ctr:ctr + 1]
                    chan = Ln

                    ctr = ctr + 2

                    # ---- channel voice messages:
                elif __switch_0__ == 'channel_voice':

                    type = bitshift(Hn, 4)
                    len = channel_voice_msg_len(type)            # var length data:
                    thedata = track[ctr:ctr + len - 1]
                    chan = Ln

                    # DEBUG:
                    if (i == 2):
                        if (msgCtr == 1):
                            disp([999, Hn, type])



                    ctr = ctr + len

                elif __switch_0__ == 'sysex':

                    # UNSURE: do sysex events (F0-F7) have
                    #  variable length 'length' field?

                    [len, ctr] = decode_var_length(track, ctr)

                    type = B
                    thedata = track[ctr:ctr + len - 1]
                    chan = []

                    ctr = ctr + len

                elif __switch_0__ == 'sys_realtime':

                    # UNSURE: I think these are all just one byte
                    type = B
                    thedata = []
                    chan = []



                last_byte = Ln + bitshift(Hn, 4)

            # end midi event 'if'


            currMsg.deltatime = deltatime
            currMsg.midimeta = midimeta
            currMsg.type = type
            currMsg.data = thedata
            currMsg.chan = chan

            if (rawbytes):
                currMsg.rawbytes = track[ctr_start_msg:ctr - 1]

            midi.track(i).messages(msgCtr).lvalue = currMsg
            msgCtr = msgCtr + 1


    # end loop over rawbytes
    # end loop over tracks


    def decode_int(A=None):

        val = 0
        for i in range(1,len(A)):
            val = val + bitshift(A(len(A) - i + 1), 8 * (i - 1))



    def channel_voice_msg_len(type=None):

        if (type == 128):
            len = 2

        elif (type == 144):
            len = 2

        elif (type == 160):
            len = 2

        elif (type == 176):
            len = 2

        elif (type == 192):
            len = 1

        elif (type == 208):
            len = 1

        elif (type == 224):
            len = 2

        else:
            print 'bad channel voice message type'




            #
            # decode variable length field (often deltatime)
            #
            #  return value and new position of pointer into 'bytes'
            #

    def decode_var_length(bytes=None, ptr=None):

        keepgoing = 1
        binarystring = ''
        while (keepgoing):
            # check MSB:
            #  if MSB=1, then delta-time continues into next byte...
            if (not bitand(bytes(ptr), 128)):
                keepgoing = 0

            # keep appending last 7 bits from each byte in the deltatime:
            binbyte = ['00000000', dec2base(bytes(ptr), 2)]
            binarystring = [binarystring, binbyte[end - 6:end]]
            ptr = ptr + 1

        val = base2dec(binarystring, 2)




        #
        # Read first 2 bytes of msg and
        #  determine the type
        #  (most require only 1st byte)
        #
        # str is one of:
        #  'channel_mode'
        #  'channel_voice'
        #  'sysex'
        #  'sys_realtime'
        #

    def midi_msg_type(B=None, nB=None):

        Hn = bitshift(B, -4)
        Ln = bitand(B, 7)

        # ---- channel mode messages:
        #if (Hn==11 && nB>=120 && nB<=127)
        if (Hn == 11 and nB >= 122 and nB <= 127):
            str = 'channel_mode'

            # ---- channel voice messages:
        elif (Hn >= 8 and Hn <= 14):
            str = 'channel_voice'

            #  ---- sysex events:
        elif (Hn == 15 and Ln >= 0 and Ln <= 7):
            str = 'sysex'

            # system real-time messages
        elif (Hn == 15 and Ln >= 8 and Ln <= 15):
            # UNSURE: how can you tell between 0xFF system real-time
            #   message and 0xFF meta event?
            #   (now, it will always be processed by meta)
            str = 'sys_realtime'

        else:
            # don't think it can get here...
            print 'bad midi message'


IndentationError: expected an indented block (<ipython-input-14-9efd1da363be>, line 112)

In [4]:
n = [1, 2, 3, 4, 5]
print n[:,5]


TypeError: list indices must be integers, not tuple